# CIFAR-10


Final Accuracy: **90.35%** at 47 epochs


In [0]:
# https://keras.io/
!pip install -q keras
import keras

Using TensorFlow backend.


In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 64
num_classes = 10
epochs = 50
l = 40
num_filter = 64
compression = 0.5
dropout_rate = 0.3

In [0]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoding 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

170500096/170498071 [==============================] - 26s 0us/step


In [0]:
# Dense Block
def add_denseblock(input, num_filter, dropout_rate):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter, dropout_rate):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_filter = 64
dropout_rate = 0.2
l = 12
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Fourth_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Fourth_Transition = add_transition(Fourth_Block, num_filter, dropout_rate)

Fifth_Block = add_denseblock(Fourth_Transition, num_filter, dropout_rate)
Fifth_Transition = add_transition(Fifth_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Fifth_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)

In [0]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 64)   1728        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 64)   256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 64)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

conv2d_69 (Conv2D)              (None, 2, 2, 32)     27648       activation_68[0][0]              
__________________________________________________________________________________________________
dropout_68 (Dropout)            (None, 2, 2, 32)     0           conv2d_69[0][0]                  
__________________________________________________________________________________________________
concatenate_63 (Concatenate)    (None, 2, 2, 128)    0           concatenate_62[0][0]             
                                                                 dropout_68[0][0]                 
__________________________________________________________________________________________________
batch_normalization_69 (BatchNo (None, 2, 2, 128)    512         concatenate_63[0][0]             
__________________________________________________________________________________________________
activation_69 (Activation)      (None, 2, 2, 128)    0           batch_normalization_69[0][0]     
__________

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [0]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 662s 13ms/step - loss: 1.4568 - acc: 0.4643 - val_loss: 2.0191 - val_acc: 0.4783
Epoch 2/50
50000/50000 [==============================] - 642s 13ms/step - loss: 0.9996 - acc: 0.6465 - val_loss: 1.7876 - val_acc: 0.5161
Epoch 3/50
50000/50000 [==============================] - 637s 13ms/step - loss: 0.7969 - acc: 0.7194 - val_loss: 1.5678 - val_acc: 0.5793
Epoch 4/50
50000/50000 [==============================] - 638s 13ms/step - loss: 0.6805 - acc: 0.7620 - val_loss: 1.0285 - val_acc: 0.6815
Epoch 5/50
50000/50000 [==============================] - 639s 13ms/step - loss: 0.6042 - acc: 0.7919 - val_loss: 0.8310 - val_acc: 0.7461
Epoch 6/50
50000/50000 [==============================] - 637s 13ms/step - loss: 0.5442 - acc: 0.8135 - val_loss: 0.9614 - val_acc: 0.7100
Epoch 7/50
50000/50000 [==============================] - 639s 13ms/step - loss: 0.4906 - acc: 0.8305 - val_loss: 0.

In [0]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=30,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/30
50000/50000 [==============================] - 654s 13ms/step - loss: 0.4246 - acc: 0.8539 - val_loss: 1.8631 - val_acc: 0.5898
Epoch 2/30
50000/50000 [==============================] - 637s 13ms/step - loss: 0.3897 - acc: 0.8651 - val_loss: 0.7877 - val_acc: 0.7769
Epoch 3/30
50000/50000 [==============================] - 637s 13ms/step - loss: 0.3509 - acc: 0.8785 - val_loss: 0.7787 - val_acc: 0.7758
Epoch 4/30
50000/50000 [==============================] - 636s 13ms/step - loss: 0.3313 - acc: 0.8839 - val_loss: 0.7873 - val_acc: 0.7878
Epoch 5/30
50000/50000 [==============================] - 636s 13ms/step - loss: 0.3084 - acc: 0.8940 - val_loss: 1.0636 - val_acc: 0.7360
Epoch 6/30
50000/50000 [==============================] - 636s 13ms/step - loss: 0.2874 - acc: 0.8994 - val_loss: 0.4898 - val_acc: 0.8556
Epoch 7/30
50000/50000 [==============================] - 636s 13ms/step - loss: 0.2695 - acc: 0.9069 - val_loss: 0.

In [0]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=5,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/5
50000/50000 [==============================] - 648s 13ms/step - loss: 0.0816 - acc: 0.9710 - val_loss: 0.7192 - val_acc: 0.8497
Epoch 2/5
50000/50000 [==============================] - 649s 13ms/step - loss: 0.0745 - acc: 0.9733 - val_loss: 0.7022 - val_acc: 0.8634
Epoch 3/5
50000/50000 [==============================] - 649s 13ms/step - loss: 0.0790 - acc: 0.9722 - val_loss: 0.6045 - val_acc: 0.8714
Epoch 4/5
50000/50000 [==============================] - 647s 13ms/step - loss: 0.0709 - acc: 0.9746 - val_loss: 0.5276 - val_acc: 0.8867
Epoch 5/5
50000/50000 [==============================] - 647s 13ms/step - loss: 0.0694 - acc: 0.9755 - val_loss: 0.6293 - val_acc: 0.8684


In [0]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=2,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/2
50000/50000 [==============================] - 648s 13ms/step - loss: 0.0656 - acc: 0.9772 - val_loss: 0.5762 - val_acc: 0.8741
Epoch 2/2
50000/50000 [==============================] - 647s 13ms/step - loss: 0.0656 - acc: 0.9772 - val_loss: 0.5783 - val_acc: 0.8833


In [0]:
# Test the model

score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 45s 4ms/step
Test loss: 0.5783102591097354
Test accuracy: 0.8833


In [0]:
# Save the trained weights in to .h5 format
model.save_weights("Prajwal_DNST_model.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
model.load_weights('Prajwal_DNST_model.h5')

In [0]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=1,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/1
50000/50000 [==============================] - 629s 13ms/step - loss: 0.0615 - acc: 0.9785 - val_loss: 0.5474 - val_acc: 0.8850


In [0]:
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 45s 5ms/step
Test loss: 0.5474003221592866
Test accuracy: 0.885


In [0]:
# Changed the batch size to 128 from 64

model.fit(x_train, y_train,
                    batch_size=128,
                    epochs=1,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/1
50000/50000 [==============================] - 524s 10ms/step - loss: 0.0242 - acc: 0.9923 - val_loss: 0.5090 - val_acc: 0.9035


In [0]:
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 44s 4ms/step
Test loss: 0.5090395306722261
Test accuracy: 0.9035


In [0]:
# Save the trained weights in to .h5 format
model.save_weights("Final_DNST_model.h5")
print("Saved model to disk")

Saved model to disk
